In [ ]:
import re
import os
import time
from shutil import copyfile
from tqdm.notebook import tqdm

import pathlib
from pathlib import Path

import pandas as pd
import numpy as np
from math import ceil

from IPython.display import display, Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

import cv2
from PIL import Image


import librosa
import librosa as lb
import soundfile as sf

import torch
import torch.optim as optim
from torch import nn
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch.utils.data import Dataset, DataLoader
from torchvision.datasets import ImageFolder
from torchvision.transforms import Resize,transforms, ToTensor

!pip install git+https://github.com/ulaval-damas/glo4030-labs.git
from deeplib.history import History
from deeplib.datasets import train_valid_loaders
import torchvision.models as models
from deeplib.visualization import plot_images

from poutyne import Experiment
from sklearn.metrics import accuracy_score
#from sklearn.metrics import top_k_accuracy_score


# For use TPU
#!pip install cloud-tpu-client==0.10 https://storage.googleapis.com/tpu-pytorch/wheels/torch_xla-1.7-cp37-cp37m-linux_x86_64.whl
# import torch_xla
# import torch_xla.core.xla_model as xm
# dev = xm.xla_device()
#DEVICE = torch.device(dev if torch.xla_device(dev).is_available() else "cpu")  #"cuda" 

dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

!pip install GPUtil
from GPUtil import showUtilization as gpu_usage
gpu_usage()

import torch
torch.cuda.empty_cache()

from numba import cuda
cuda.select_device(0)
cuda.close()
cuda.select_device(0)

  Cloning https://github.com/ulaval-damas/glo4030-labs.git to /tmp/pip-req-build-2c9gu4zo
  Running command git clone -q https://github.com/ulaval-damas/glo4030-labs.git /tmp/pip-req-build-2c9gu4zo
     |████████████████████████████████| 143kB 5.2MB/s 
  Created wheel for deeplib: filename=deeplib-0.1-cp37-none-any.whl size=10554 sha256=a433d4edf8e7a7d3c2afa598b12bd832b404a9bd64eac45ee5c36656fa40098b
  Stored in directory: /tmp/pip-ephem-wheel-cache-vkozlk34/wheels/5e/d1/5f/50a2caa0ee64b5f4a2ed0d59696c01fe8f4f467123ff81095f
Successfully built deeplib
  Created wheel for GPUtil: filename=GPUtil-1.4.0-cp37-none-any.whl size=7411 sha256=271a9a16392ab7c34c753c38be1fff9d03822f5d79e417654405dfba9e3831e9
  Stored in directory: /root/.cache/pip/wheels/3d/77/07/80562de4bb0786e5ea186911a2c831fdd0018bda69beab71fd
Successfully built GPUtil
| ID | GPU | MEM |
------------------
|  0 |  0% |  0% |


<weakproxy at 0x7f6a4dfd0d10 to Device at 0x7f6a565787d0>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# FONCTION UTILITAIRES

Tirée du notebook : https://www.kaggle.com/kneroma/clean-fast-simple-bird-identifier-inference

In [ ]:
class EarlyStopping:
    """Early stops the training if validation loss doesn't improve after a given patience."""
    def __init__(self, patience=3, verbose=False, delta=0, path='checkpoint.pt', trace_func=print):
        """
        Args:
            patience (int): How long to wait after last time validation loss improved.
                            Default: 7
            verbose (bool): If True, prints a message for each validation loss improvement. 
                            Default: False
            delta (float): Minimum change in the monitored quantity to qualify as an improvement.
                            Default: 0
            path (str): Path for the checkpoint to be saved to.
                            Default: 'checkpoint.pt'
            trace_func (function): trace print function.
                            Default: print            
        """
        self.patience = patience
        self.verbose = verbose
        self.counter = 0
        self.best_score = None
        self.early_stop = False
        self.val_loss_min = np.Inf
        self.delta = delta
        self.path = path
        self.trace_func = trace_func
    def __call__(self, val_loss, model):

        score = -val_loss

        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            self.trace_func(f'EarlyStopping counter: {self.counter} out of {self.patience}')
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        '''Saves model when validation loss decrease.'''
        if self.verbose:
            self.trace_func(f'Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}).  Saving model ...')
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss


# Fonctions pour transférer un audio en image
def get_spectrograms(filepath,savepath,k):
    
    # Open the file with librosa (limited to the first 15 seconds)
    sig, rate = librosa.load(filepath, sr=SAMPLE_RATE, offset=None, duration=15)
    
    # Split signal into five second chunks
    sig_splits = []
    for i in range(0, len(sig), int(SIGNAL_LENGTH * SAMPLE_RATE)):
        split = sig[i:i + int(SIGNAL_LENGTH * SAMPLE_RATE)]

        # End of signal?
        if len(split) < int(SIGNAL_LENGTH * SAMPLE_RATE):
            break
        
        sig_splits.append(split)
        
    # Extract mel spectrograms for each audio chunk
    j = 0
    saved_samples = []
    for chunk in sig_splits:
        
        # Spectogramm
        hop_length = int(SIGNAL_LENGTH * SAMPLE_RATE / (SPEC_SHAPE[1] - 1))
        mel_spec = librosa.feature.melspectrogram(y=chunk, 
                                                  sr=SAMPLE_RATE, 
                                                  n_fft=1024, 
                                                  hop_length=hop_length, 
                                                  n_mels=SPEC_SHAPE[0], 
                                                  fmin=FMIN, 
                                                  fmax=FMAX)
    
        mel_spec = librosa.power_to_db(mel_spec, ref=np.max) 
        
        # Amplitude
        # BINS_PER_OCTAVE = 12 * 3
        # N_OCTAVES = 7
        # mel_amp = librosa.amplitude_to_db(np.abs(librosa.cqt(y=sig, sr=rate,
        #                                 bins_per_octave=BINS_PER_OCTAVE,
        #                                 n_bins=N_OCTAVES * BINS_PER_OCTAVE)),
        #                                 ref=np.max)
        
        # display
        # fig, ax = plt.subplots()
        # librosa.display.specshow(mel_amp, y_axis='cqt_hz', sr=rate,
        #                         bins_per_octave=BINS_PER_OCTAVE,
        #                         x_axis='time', ax=ax)

        # Normalize
        mel_spec -= mel_spec.min()
        if mel_spec.max()!= 0:
            mel_spec /= mel_spec.max()
        else:
            mel_spec /= 0.00001
        
        # Save as image file
#         save_dir = os.path.join(output_dir, primary_label)
#         if not os.path.exists(save_dir):
#             os.makedirs(save_dir)
#         save_path = os.path.join(save_dir, filepath.rsplit(os.sep, 1)[-1].rsplit('.', 1)[0] + 
#                                  '_' + str(s_cnt) + '.png')

        im = Image.fromarray(mel_spec * 255.0).convert("1")
        save_path = savepath +'/'+ str(k)+'_'+ str(j) +'.jpg'
        im.save(save_path)
        imsave('result_col.png', im)
        
#         saved_samples.append(save_path)
        j += 1
        
        
    return im


def make_dir(file_path):
    os.makedirs(file_path)



def separate_train_test(dataset_path, train_path, test_path):
    """
    Cette fonction sépare les images de CUB200 en un jeu d'entraînement et de test.

    dataset_path: Path où se trouve les images de CUB200
    train_path: path où sauvegarder le jeu d'entraînement
    test_path: path où sauvegarder le jeu de test
    """
    class_index = 1
    for classname in sorted(os.listdir(dataset_path)):
        print(classname)
        make_dir(os.path.join(train_path, classname))
        make_dir(os.path.join(test_path, classname))
        i = 0
        
        nb_file = len(os.listdir(os.path.join(dataset_path, classname)))
        nb_test = ceil(nb_file*0.2)
       
        for file in sorted(os.listdir(os.path.join(dataset_path, classname))):
            
            file_path = os.path.join(dataset_path, classname, file)
            
            if i < nb_test:
                get_spectrograms(file_path,os.path.join(test_path, classname),i)
                
            else:
                get_spectrograms(file_path,os.path.join(train_path, classname),i)
                
            i += 1

#         class_index += 1


# VARIABLES GLOBALES

In [ ]:
nb_classes = 397

# spectogram
SAMPLE_RATE = 32_000
SIGNAL_LENGTH= 5
SPEC_SHAPE = (48, 128)
FMIN = 500
FMAX = 12500

transform = transforms.Compose(
        [transforms.Resize([448, 448]),
         transforms.ToTensor()]) 

DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# test_audio_path = Path("C:/Users/isabelle/Desktop/DL/FINAL/birdclef-2021/test_soundscapes")
# train_audio_path = Path("C:/Users/isabelle/Desktop/DL/FINAL/birdclef-2021/train_soundscapes")
# train_short_audio_path = ("C:/Users/isabelle/Desktop/DL/FINAL/birdclef-2021/train_short_audio")
# label_path = Path("C:/Users/isabelle/Desktop/DL/FINAL/birdclef-2021/train_soundscape_labels.csv")

train_path = "./kaggle/working/train_img"
test_path  = "./kaggle/working/test_img"
dataset_path = "./MyDrive/data/train_short_audio/"

# separate_train_test(dataset_path,train_path,test_path)
# !zip -r train.zip "/content/drive/MyDrive/Birds/train (2)"
# !zip -r test.zip "/content/drive/MyDrive/Birds/test (1)"

!unzip -qq "/content/drive/MyDrive/Birds/train (2)"
!unzip -qq "/content/drive/MyDrive/Birds/test (1)"
print('Done')

train_data = ImageFolder('./kaggle/working/train_img',transform=transform)
test_data = ImageFolder('./kaggle/working/test_img',transform=transform)
print("ImageFolder 1...")


# Images Noir & Blanc
print("Images Noir & Blanc")
from pathlib import Path
from typing import Callable
from tqdm import tqdm

base_path_train = Path("./kaggle/working/train_img")
base_path_test = Path("./kaggle/working/test_img")

def grey_to_black(origin_image_path, new_image_path):
    im_gray = cv2.imread(origin_image_path, cv2.IMREAD_GRAYSCALE)
    (thresh, im_bw) = cv2.threshold(im_gray, 128, 255, cv2.THRESH_BINARY | cv2.THRESH_OTSU)
    im_bw = cv2.threshold(im_gray, thresh, 255, cv2.THRESH_BINARY)[1]
    cv2.imwrite(new_image_path, im_bw)

def edit_images(base_path: Path, folder_to_switch: str, new_folder_name: str, filter_fct: Callable):
    for file in tqdm(base_path.glob('**/*')):
        if file.is_file():
            new_path = Path(str(file).replace(folder_to_switch, new_folder_name))
            if not new_path.parent.exists():
                new_path.parent.mkdir(parents=True)
            filter_fct(str(file), str(new_path))

edit_images(base_path=base_path_train, folder_to_switch='train_img', new_folder_name='train_img_2', filter_fct=grey_to_black)
edit_images( base_path=base_path_test, folder_to_switch='test_img', new_folder_name='test_img_2', filter_fct=grey_to_black)

train_data = ImageFolder('./kaggle/working/train_img_2/',transform=transform)
test_data = ImageFolder('./kaggle/working/test_img_2/',transform=transform)
print("ImageFolder 2...")

Done


536it [00:00, 5341.91it/s]

ImageFolder 1...
Images Noir & Blanc


137724it [00:53, 2569.73it/s]
35285it [00:13, 2609.24it/s]


ImageFolder 2...


# Entrainement du modèle

In [ ]:


def train(model, dataset, n_epoch, batch_size, learning_rate, use_gpu=False):
    """
    Entraîne un réseau de neurones de classification pour un certain nombre d'epochs 
    avec PyTorch.
    
    Args:
        model (nn.Module): Un réseau de neurones instancié avec PyTorch.
        dataset (Dataset): Un jeu de données PyTorch.
        n_epoch (int): Le nombre d'epochs.
        batch_size (int): La taille des batchs.
        learning_rate (float): Le taux d'apprentissage pour SGD.
        use_gpu (bool): Si les données doivent être envoyées sur GPU.
    
    Returns:
        Retourne un objet History permettant de faire des graphiques
        de l'évolution de l'entraînement.
    """
    # La classe History vient de deeplib. Elle va nous permettre de faire les graphiques
    # donnant l'évolution de la perte et de l'exactitude (accuracy).
    history = History()
    
    # La fonction de perte que nous utilisons ici est l'entropy croisée
    # L'optimiseur que nous utilisons ici est le classique SGD.
    # Des liens vers la documentation de PyTorch sont en commentaires.
    criterion = nn.CrossEntropyLoss()  # https://pytorch.org/docs/stable/generated/torch.nn.CrossEntropyLoss.html#torch.nn.CrossEntropyLoss
    optimizer = optim.Adam(model.parameters(), lr=learning_rate)  # https://pytorch.org/docs/stable/optim.html#torch.optim.SGD
    scheduler = ReduceLROnPlateau(optimizer =optimizer, patience = 4, verbose = True)
    early_stopping = EarlyStopping(patience=10, verbose=True)
    
    # La fonction train_valid_loaders vient de deeplib. Elle nous retourne deux DataLoaders:
    # un pour l'ensemble d'entraînement et un pour l'ensemble de test. Essentiellement,
    # DataLoader est une classe de PyTorch nous permettant de faire des batchs avec la taille
    # désirée. La fonction train_valid_loaders effectue la répartition aléatoire des exemples
    # en entraînement et en validation.
    train_loader, valid_loader = train_valid_loaders(dataset, batch_size=batch_size)

    # C'est ici que la boucle d'entraînement commence. On va donc faire n_epochs epochs.
    for i in range(n_epoch):
        torch.cuda.empty_cache()
        # Les réseaux de neurones avec PyTorch ont un méthode train() en plus d'une méthode 
        # eval(). Ces deux méthodes indiquent au réseau s'il est en entraînement ou bien en test.
        # Ceci permet au réseau de modifier son comportement en fonction. On va le voir plus tard
        # certaines couches agissent différemment selon le mode, nommément le dropout et la 
        # batch normalization.
        model.train()
        
        # La prochaine ligne active simplement le calcul du gradient. Le gradient est ce qui va
        # nous permettre de mettre à jour les poids du réseau de neurones. En test, le calcul du
        # gradient sera désactivé étant qu'il n'est pas nécessaire et qu'il peut engendrer des 
        # fuites de mémoire si la rétro-propagation n'est pas effectuée.
        with torch.enable_grad():
            # À chaque epoch, on parcourt l'ensemble d'entraînement au complet via le DataLoader
            # qui nous le retourne en batch (x, y) comme mentionné plus haut. La variable inputs 
            # correspond donc à une batch d'exemples (x) et targets correspond à une batch 
            # d'étiquettes (y).
            for inputs, targets in train_loader:
                # On envoie les exemples et leurs étiquettes sur GPU via la méthode cuda() si 
                # demandé.
                if use_gpu:
                    inputs = inputs.cuda()
                    targets = targets.cuda()

                # La méthode zero_grad() de l'optimiseur permet de mettre la valeur du gradient
                # à zéro de façon à effacer le gradient calculé auparavant. Si ceci n'était pas 
                # fait, le nouveau gradient serait additionné à l'ancien gradient ce qui poserait
                # problème.
                optimizer.zero_grad()
                
                # C'est ici que finalement le réseau de neurones est appelé. On lui donne en entrée
                # un exemple et en sortie il nous donne ses prédictions (ici, des scores de 
                # classification).
                output = model(inputs)

                # Une fois nos prédictions obtenues, on calcule la perte avec la fonction de perte 
                # qui nous retourne un tenseur scalaire. 
                loss = criterion(output, targets)
                # Ce tenseur scalaire nous permet de calculer le gradient. C'est ce que la méthode
                # backward() vient faire pour nous via la rétropropagation.
                loss.backward()
                # Une fois que le gradient est calculé, il ne reste que mettre à jour les poids du
                # réseau de neurones. C'est ce que la méthode step() de l'optimiseur vient faire
                # pour nous.
                optimizer.step()
                #del loss, output

        # Après chaque epoch d'entraînement, on va venir calculer la perte et l'exactitude 
        # (accuracy) sur l'ensemble d'entraînement et de validation.
        train_acc, train_loss = validate(model, train_loader, use_gpu)
        val_acc, val_loss = validate(model, valid_loader, use_gpu)
        early_stopping(val_loss, model)
        
        history.save(dict(acc=train_acc, val_acc=val_acc, loss=train_loss, val_loss=val_loss, lr=learning_rate))
        if early_stopping.early_stop:
            print("Early stopping")
            break
        print(f'Epoch {i} - Train acc: {train_acc:.2f} - Val acc: {val_acc:.2f} - Train loss: {train_loss:.4f} - Val loss: {val_loss:.4f}')
    model.load_state_dict(torch.load('checkpoint.pt'))
    return history, model


def validate(model, valid_loader, use_gpu=False):
    """
    Test un réseau de neurones de classification pour un certain nombre d'epochs 
    avec PyTorch.
    
    Args:
        model (nn.Module): Un réseau de neurones instancié avec PyTorch.
        valid_loader (DataLoader): Un DataLoader PyTorch tel qu'instancié dans train() 
            et test().
        use_gpu (bool): Si les données doivent être envoyées sur GPU.
            
    Returns:
        Retourne un tuple (exactitude, perte) pour les données du DataLoader en argument.
    """
    
    # Les étapes de la fonction validate est très similaire à celle de la fonction train.
    # Essentiellement, le réseau est mis en mode évaluation au lieu d'entraînement et le 
    # calcul du gradient est désactivé. Il n'y a bien sûr pas d'utilisation d'un optimiseur.
    true = []
    pred = []
    val_loss = []
    
    criterion = nn.CrossEntropyLoss()
    model.eval()
    with torch.no_grad():
        for inputs, targets in valid_loader:
            if use_gpu:
                inputs = inputs.cuda()
                targets = targets.cuda()

            output = model(inputs)

            predictions = output.max(dim=1)[1]

            val_loss.append(criterion(output, targets).item())
            true += targets.cpu().numpy().tolist()
            pred += predictions.cpu().numpy().tolist()

    return accuracy_score(true, pred) * 100, sum(val_loss) / len(val_loss)


def test(model, dataset, batch_size, use_gpu=False):
    """
    Test un réseau de neurones de classification pour un certain nombre d'epochs 
    avec PyTorch. La fonction affiche l'exactitude et la perte moyenne.
    
    Args:
        model (nn.Module): Un réseau de neurones instancié avec PyTorch.
        dataset (Dataset): Un jeu de données PyTorch.
        batch_size (int): La taille des batchs.
        use_gpu (bool): Si les données doivent être envoyées sur GPU.
    """
    test_loader = DataLoader(dataset, batch_size=batch_size)
    test_acc, test_loss = validate(model, test_loader, use_gpu=use_gpu)
    print('Test acc: {:.2f} - Test loss: {:.4f}'.format(test_acc, test_loss))



# PROGRAMME

In [ ]:
# Architecture du reseau
# resnet18 = models.resnet18(pretrained = True)
# ct = 0
# for child in resnet18.children():
#     print(ct)
#     print(child)

#     ct+=1

# Model ResNet
#nb_classes = 397
#model = models.resnet34(pretrained=False)
# ct = 0
# for child in resnet18.children():
#     if ct <= 4:
#         for param in child.parameters():
#             param.requires_grad = False
#     ct+=1
#model.fc = nn.Linear(model.fc.in_features, nb_classes)

# Model VGG
# model = models.vgg16(pretrained=False)
# num_ftrs = model.classifier[0].in_features
# model.classifier[0].out_features = nn.Linear(num_ftrs, nb_classes)

!pip install GPUtil
from GPUtil import showUtilization as gpu_usage
gpu_usage()

# !pip install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp37-cp37m-linux_x86_64.whl
# import torch
# torch.cuda.empty_cache()

# from numba import cuda
# cuda.select_device(0)
# cuda.close()
# cuda.select_device(0)

#gpu_usage()

# def free_gpu_cache():
#     print('Initial GPU Usage')
#     gpu_usage()

#     torch.cuda.empty_cache()

#     cuda.select_device(0)
#     cuda.close()
#     cuda.select_device(0)

#     print('GPU Usage after emptying the cache')

#     gpu_usage()

#free_gpu_cache()


#import logging
#torch.cuda.empty_cache()
#torch.cuda.memory_summary()

# import gc
# del model, 
# gc.collect()
# torch.cuda.empty_cache()

#try:
nb_classes = 397
epoch = 20
batch_size = 256
learning_rate = 0.01

model = models.resnet34(pretrained=False)
model.fc = nn.Linear(model.fc.in_features, nb_classes)

if torch.cuda.is_available():
    model.cuda()

history, model = train(model, train_data, epoch, batch_size, learning_rate, use_gpu=True)
history.display()
test(model, test_data, batch_size, use_gpu=True)

torch.cuda.empty_cache()



#except BaseException as err:
    #logging.exception(err)
    #torch.cuda.empty_cache()

#finally:
    #torch.cuda.empty_cache()

| ID | GPU | MEM |
------------------
|  0 |  0% |  5% |
     |████████████████████████████████| 770.7MB 20kB/s 
ERROR: torchvision 0.9.1+cu101 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
ERROR: torchtext 0.9.1 has requirement torch==1.8.1, but you'll have torch 1.1.0 which is incompatible.
  Found existing installation: torch 1.8.1+cu101
    Uninstalling torch-1.8.1+cu101:
      Successfully uninstalled torch-1.8.1+cu101


RuntimeError: ignored

In [ ]:
# history.(dict(acc=train_acc, val_acc=val_acc, loss=train_loss, val_loss=val_loss, lr=learning_rate))
# score = test(model, test_data, batch_size, use_gpu=True)
# top_k_accuracy_score(y_true=history.acc, y_score=score, 10, normalize=False)

In [ ]:
# nb_classes = 397
# resnet50= models.resnet50(pretrained = False)

# resnet50.fc = nn.Linear(resnet50.fc.in_features, nb_classes)
# epoch = 30
# batch_size = 16
# learning_rate = 0.001
# if torch.cuda.is_available():
#   resnet50.cuda()
#history,resnet50 = train(resnet50, train_data, epoch, batch_size, learning_rate, use_gpu=True)

In [ ]:
# import numpy as np
# import seaborn as sns
# import matplotlib.pyplot as plt
# from tqdm import tqdm
# from tqdm import tqdm
# import warnings

# warnings.filterwarnings(action='ignore')



# # Just Seaborn Barplot w/ Common Input Format
# def bar_plot(x, y, title='Training Soundscape : 5 second segment identification', xlim=None, ylim=None,
#              xticklabels=None, yticklabels=None, xlabel=None, ylabel=None,
#              figsize=(10, 4), axis_grid='x', xrotation=None, yrotation=None):

#     cmap = sns.color_palette("mako")
#     fig, ax = plt.subplots(figsize=figsize)
#     plt.title(title)

#     for i in ['top', 'right', 'bottom', 'left']:
#         ax.spines[i].set_color('black')

#     ax.spines['top'].set_visible(True);
#     ax.spines['right'].set_visible(False)
#     ax.spines['bottom'].set_visible(False);
#     ax.spines['left'].set_visible(False)

#     sns.barplot(x=x, y=y, edgecolor='black', ax=ax, palette=cmap)
#     ax.set_xlim(xlim);
#     ax.set_ylim(ylim)
#     #     ax.set_xticklabels(xticklabels);ax.set_yticklabels(yticklabels)
#     plt.xlabel(xlabel);
#     plt.ylabel(ylabel)
#     ax.grid(axis=axis_grid, ls='--', alpha=0.3)
#     plt.xticks(rotation=xrotation)
#     plt.yticks(rotation=yrotation)
#     plt.savefig('bar_out.png');
#     plt.show();

# # Visualise the Birds present in Recordings
# bar_plot(x=x[1:], y=y[1:], figsize=(20, 5), xrotation=90, title='Soundscape Data: Birds Labeled in All Recordings')